In [ ]:
#Libraries
import pandas as pd #Bibliothek für Datenmanipulation und -analyse.
import numpy as np #Bibliothek für mathematische Funktionen.
import matplotlib.pyplot as plt #Bibliothek zum Erstellen von statischen, animierten und interaktiven Visualisierungen.
import seaborn as sns #Bibliothek für statistische Datenvisualisierung basierend auf matplotlib.
# knn
from sklearn.neighbors import KNeighborsClassifier #K-Nearest Neighbors Klassifikator aus scikit-learn.
from sklearn import neighbors #Modul für K-Nearest Neighbors Algorithmen.
from sklearn.model_selection import cross_val_score #Funktion zur Bewertung eines Scores durch Kreuzvalidierung.
from sklearn.preprocessing import StandardScaler #Standardisiert Merkmale, indem der Mittelwert entfernt und auf die Einheitsskala skaliert wird.
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV #Funktion zum Aufteilen von Arrays oder Matrizen in zufällige Trainings- und Testsätze.
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay #Modul zur Berechnung von Klassifikationsmetriken.
from mlxtend.plotting import plot_decision_regions #Funktion zum Plotten von Entscheidungsregionen von Klassifikatoren.
# Entscheidungsbaum
from sklearn.tree import DecisionTreeClassifier #Entscheidungsbaum-Klassifikator aus scikit-learn.
# Random Forest
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier #Random Forest Klassifikator aus scikit-learn.
#Neurnales Netz
from tensorflow.keras.models import Sequential #Sequenzielles Modell aus Keras, einer High-Level-API für neuronale Netze.
from tensorflow.keras.utils import to_categorical #Funktion zur Umwandlung eines Klassenvektors (Ganzzahlen) in eine binäre Klassenmatrix.
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization #Dichte (vollständig verbundene) Schicht aus Keras.
from tensorflow.keras.callbacks import EarlyStopping #Callback zum Stoppen des Trainings, wenn eine überwachte Metrik sich nicht mehr verbessert.


# CSV-Datei einlesen
telefonkunden = pd.read_csv('telefonkunden.csv')

# Wie sieht der Datensatz aus?
print(telefonkunden.head)
telefonkunden.shape

# - 2. den Datensatz in Trainings- und Lerndatensatz aufteilen

# Dafür erstellen wir zunächst (wie auch bei den Weinen) zwei separate Datensätze mit den Input- (X) und der Outputvariablen (Y).
X = telefonkunden.drop(columns=['custcat'])
Y = telefonkunden.custcat
X.head()
Y.head()

# Jetzt können wir wieder mit train_test_split aus scikit-learn den Datensatz in Trainings- und Testdaten aufteilen

X_training, X_test, Y_training, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1, stratify = Y)
# Wie vorher: Wir teilen in 20% Lern- und 80% Testdaten auf (test_size), lassen die Aufteilung zur Reproduzierbarkeit der Ergebnisse immer gleich aufteilen (random_state) und sorgen dafür, dass die Klassen gleichmäßig aufgeteilt sind (stratify).

# Nun wird das Modelll trainiert. Wir verwenden zunächst k=5 Nachbarn.
knn = KNeighborsClassifier(n_neighbors = 5)
vorhersage_scikit = knn.fit(X_training, Y_training)

# - 3. eine Vorhersage mit dem kNN-Algorithmus treffen
vorhersage_scikit=knn.predict(X_test)
#Ergebnis 
knn.score(X_test, Y_test)
# Die bisherige Vorgehensweise hieß Holdout-Methode: Man reserviert eine Menge als Testdaten, den Rest als Trainingsdaten. Allerdings kann dies zu Probleme führen: 
# Vielleicht wurden die Daten unglücklich aufgeteilt?Zur Lösung bedient man sich der sog. k-fache Kreuzvalidierung, wie auch in der Vorlesung besprochen: 
# Bei der Kreuzvalidierung wird der Datensatz zufällig in Gruppen aufgeteilt. Eine der Gruppen wird als Testsatz und der Rest als Trainingssatz verwendet. 
# Das Modell wird mit dem Trainingssatz trainiert und mit dem Testsatz bewertet. Dann wird der Prozess wiederholt, bis jede einzelne Gruppe als Testsatz verwendet wurde. 
# Bei der 5-fachen Kreuzvalidierung werden also die Daten in 5 Gruppen aufgeteilt und 5-mal angepasst und bewertet, wobei jedes Mal der Genauigkeitswert in einem Array abgespeichert wird. So hat jede der 5 Gruppen eine Chance, als Testdatensatz zu dienen.
# Neuen Klassifikator verwenden, mit unserem idealen k=5
klassifikator_kreuzvalidierung = KNeighborsClassifier(n_neighbors=13)

# Genauigkeiten mit einem trainierten Modell mit 5 Gruppen
kreuzvalidierung_genauigkeiten = cross_val_score(klassifikator_kreuzvalidierung, X, Y, cv = 5)

# Genauigkeiten ausgeben
print(kreuzvalidierung_genauigkeiten)
# Nehmen wir einfach mal den Mittelwert dieser Genauigkeiten, und sehen, dass wir die Genauigkeit nochmals erhöhen konnten von ca. 66% auf ca. 71%!
print(np.mean(kreuzvalidierung_genauigkeiten))
# Wir konnten also eine Erhöhung der Genauigkeit erreichen!
# Definieren wir zunächst einen neuen Klassifikator:
knn2 = KNeighborsClassifier()

# Nun erstellen wir das Dictionary mit den k-Werten für die Hyperparameter-Optimimerung, die uns interessieren:
k_grid = {'n_neighbors': np.arange(1, 50)}

# Nun lassen wir die Grid-Suche für jedes k durchlaufen, jeweils mit einer 5-fachen Kreuzvalidierung:
knn_grid = GridSearchCV(knn2, k_grid, cv = 5)

# Nun trainieren wir das Modell mit den Daten und den darüber definierten Parametern:
knn_grid.fit(X, Y)

# Was ist nun die beste Anzahl an Nachbarn?
print(knn_grid.best_params_)

# Der Genauigkeits-Score bei dieser Anzahl an Nachbarn ist nochmal höher
knn_grid.best_score_
# Man kann die Funktion auch alle Genauigkeits-Scores ausgeben lassen. Dies sind also für jedes k die durchschnittlichen Werte der Genauigkeits-Scores aus je allen 5 Durchläufen der Kreuzvalidierung!
scores=knn_grid.cv_results_['mean_test_score']
print(scores)

# Wir definieren eine Bildgröße
plt.figure(figsize=(10,6))
# Wir plotten auf der x-Achse von 1 bis 49 (also die k's die wir oben in der for-Schleife durchprobiert haben)
plt.plot(range(1, len(scores) + 1), scores)
# Wir beschriften noch die Achsen
plt.title('Genauigkeit vs. k')
plt.xlabel('k')
plt.ylabel('Genauigkeit')

# Wahrheitsmatrix für das KNN-Modell
conf_matrix = confusion_matrix(Y_test, vorhersage_scikit)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=knn.classes_, yticklabels=knn.classes_)
plt.xlabel('Vorhergesagte Labels')
plt.ylabel('Wahre Labels')
plt.title('Wahrheitsmatrix für KNN-Modell')
plt.show()